In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv(r"/content/Churn_Modelling.csv")

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
df.drop("Surname",axis=1,inplace=True)

In [8]:
df.head()

,RowNumber,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
categorical_features=df[["Gender","Geography"]]

In [11]:
dummies=pd.get_dummies(data=categorical_features,drop_first=True)

In [13]:
df=pd.concat([df,dummies],axis=1)

In [15]:
df.drop(["Gender","Geography"],axis=1,inplace=True)

In [17]:
df["Exited"].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [19]:
Exited=df[df["Exited"]==1]
Otherwise=df[df["Exited"]==0]

In [21]:
print(Exited.shape)
print(Otherwise.shape)

(2037, 14)
(7963, 14)


In [24]:
Otherwise_samples=Otherwise.sample(n=2037)

In [25]:
new_data=pd.concat([Exited,Otherwise_samples],axis=0)

In [41]:
X=new_data.drop(columns="Exited",axis=1)
Y=new_data["Exited"]

In [42]:
new_data.Exited.value_counts()

1    2037
0    2037
Name: Exited, dtype: int64

In [53]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=0.8,random_state=42)

In [54]:
## LOGISTIC REGRESSION MODEL
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model_1=model.fit(X_train,Y_train)

In [56]:
model_1_pred=model_1.predict(X_test)

# New Section

In [58]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(Y_test,model_1_pred))

[[191 181]
 [169 274]]


In [59]:
print("Accuracy score for LR :",accuracy_score(Y_test,model_1_pred))

Accuracy score for LR : 0.5705521472392638


In [62]:
##NEURAL NETWORK MODEL
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
NN=MLPClassifier()

In [63]:
PARAMETERS={
     "hidden_layer_sizes" :[(12,12,2),(10,20,2),(10,10,10),(100,100)],
      "activation" : ["relu","softplus","sigmoid","linear","tanh"],
      "learning_rate_init" : [0.01,0.5,1,0.1],
      "solver":["sgd","adam"]
}

In [64]:
CLFFF=GridSearchCV(NN,PARAMETERS,cv=5)

In [65]:
CLFFF.fit(X,Y)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
480 fits failed out of a total of 800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
160 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py", line 752, in fit
    return self._fit(X, y, incremental=False)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py", line 384, in _fit
    self._validate_hyperparameters()
 

GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'activation': ['relu', 'softplus', 'sigmoid', 'linear',
                                        'tanh'],
                         'hidden_layer_sizes': [(12, 12, 2), (10, 20, 2),
                                                (10, 10, 10), (100, 100)],
                         'learning_rate_init': [0.01, 0.5, 1, 0.1],
                         'solver': ['sgd', 'adam']})

In [66]:
CLFFF.best_params_

{'activation': 'relu',
 'hidden_layer_sizes': (100, 100),
 'learning_rate_init': 0.01,
 'solver': 'adam'}

In [67]:
model_2=MLPClassifier(activation="relu",hidden_layer_sizes=(100,100),learning_rate_init=0.01,solver="adam")

In [68]:
model_2.fit(X_train,Y_train)

MLPClassifier(hidden_layer_sizes=(100, 100), learning_rate_init=0.01)

In [69]:
model_2_pred=model_2.predict(X_test)

In [70]:
confusion_matrix(Y_test,model_2_pred)

array([[331,  41],
       [366,  77]])

In [73]:
print("Accuracy Score for NN :",accuracy_score(Y_test,model_2_pred))

Accuracy Score for NN : 0.5006134969325153


In [74]:
## Decision Tree Model
from sklearn.tree import DecisionTreeClassifier
DT=DecisionTreeClassifier()

In [75]:
Parameters={
     "criterion" :["gini","entropy","information_gain"],
     "max_depth" :[1,2,3,4,5]
}

In [77]:
CLFF=GridSearchCV(DT,Parameters,cv=5)

In [78]:
CLFF.fit(X,Y)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
25 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/tree/_classes.py", line 937, in fit
    super().fit(
  File "/usr/local/lib/python3.8/dist-packages/sklearn/tree/_classes.py", line 352, in fit
    criterion = CRITERIA_CLF[self.criterion](
KeyError: 'information_gain'

  warnings.warn(some_fits_failed_message, F

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'information_gain'],
                         'max_depth': [1, 2, 3, 4, 5]})

In [79]:
CLFF.best_params_

{'criterion': 'gini', 'max_depth': 1}

In [80]:
model_3=DecisionTreeClassifier(criterion="gini",max_depth=1)

In [82]:
model_3.fit(X_train,Y_train)

DecisionTreeClassifier(max_depth=1)

In [84]:
model_3_pred=model_3.predict(X_test)

In [85]:
confusion_matrix(Y_test,model_3_pred)

array([[286,  86],
       [161, 282]])

In [87]:
print("Accuracy Score for DT:",accuracy_score(Y_test,model_3_pred))

Accuracy Score for DT: 0.6969325153374233


In [91]:
## Random Forest Model
from sklearn.ensemble import RandomForestClassifier
RF=RandomForestClassifier()

In [92]:
parameter={
    "n_estimators":[10,100,200,120,150,300]
}

In [93]:
CLFFFF=GridSearchCV(RF,parameter,cv=3)

In [94]:
CLFFFF.fit(X,Y)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [10, 100, 200, 120, 150, 300]})

In [95]:
CLFFFF.best_params_

{'n_estimators': 10}

In [96]:
model_4=RandomForestClassifier(n_estimators=10)

In [97]:
model_4.fit(X_train,Y_train)

RandomForestClassifier(n_estimators=10)

In [98]:
model_4_pred=model_4.predict(X_test)

In [99]:
confusion_matrix(Y_test,model_4_pred)

array([[314,  58],
       [146, 297]])

In [100]:
print("Accuracy Score for RF :",accuracy_score(Y_test,model_4_pred))

Accuracy Score for RF : 0.7496932515337423


In [103]:
## Naive Bayes Model
from sklearn.naive_bayes import GaussianNB
model_5=GaussianNB()

In [104]:
model_5.fit(X_train,Y_train)

GaussianNB()

In [105]:
model_5_pred=model_5.predict(X_test)

In [106]:
confusion_matrix(Y_test,model_5_pred)

array([[284,  88],
       [159, 284]])

In [107]:
print("Accuracy Score for NB :",accuracy_score(Y_test,model_5_pred))

Accuracy Score for NB : 0.6969325153374233


In [110]:
## KNN ALGORITHM
from sklearn.neighbors import KNeighborsClassifier
KNN=KNeighborsClassifier()

In [111]:
PARAmeters={
    "metric":["euclidean","hamming","manhattan"],
    "p":[1,2,3,4,5]
}

In [112]:
CL=GridSearchCV(KNN,PARAmeters,cv=5)

In [113]:
CL.fit(X,Y)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'hamming', 'manhattan'],
                         'p': [1, 2, 3, 4, 5]})

In [115]:
CL.best_params_

{'metric': 'hamming', 'p': 1}

In [117]:
model_6=KNeighborsClassifier(metric="hamming",p=1)

In [118]:
model_6.fit(X_train,Y_train)

KNeighborsClassifier(metric='hamming', p=1)

In [119]:
model_6_pred=model_6.predict(X_test)

In [120]:
confusion_matrix(Y_test,model_6_pred)

array([[263, 109],
       [147, 296]])

In [121]:
print("Accuracy Score For KNN :",accuracy_score(Y_test,model_6_pred))

Accuracy Score For KNN : 0.6858895705521473


In [122]:
MODEL ={
    "LR " : accuracy_score(Y_test,model_1_pred),
    "NN" : accuracy_score(Y_test,model_2_pred),
    "DT" : accuracy_score(Y_test,model_3_pred),
    "RF" : accuracy_score(Y_test,model_4_pred),
    "NB" : accuracy_score(Y_test,model_5_pred),
    "KNN" :accuracy_score(Y_test,model_6_pred)}

In [123]:
print(MODEL)

{'LR ': 0.5705521472392638, 'NN': 0.5006134969325153, 'DT': 0.6969325153374233, 'RF': 0.7496932515337423, 'NB': 0.6969325153374233, 'KNN': 0.6858895705521473}


---> FOR THIS PARTICULAR DATA OF CHURN MODELING RANDOM FREST IS BEST ALGORITHM